In [1]:
#Base model

import gurobipy as gp
from gurobipy import GRB

# Read instances from slo file --> length of exam period
with open('/Users/nicoleolivetto/Downloads/instances/test.slo') as file_slo:
    n_timeslots = int(file_slo.readline())  # number of time-slots
    print(n_timeslots)

# Read instances from stu file --> create two lists for studentID and examIDs they are enrolled in
StuID, Ex_enr_ID = [], []
with open('/Users/nicoleolivetto/Downloads/instances/test.stu') as stu:
    for line in stu:
        b = (line.rstrip().split())
        StuID.append(b[0])
        Ex_enr_ID.append(b[1])

# Define a dictionary to store the enrollments of each student
student_enrollments = {}

# Read the file and populate the student_enrollments dictionary
with open('/Users/nicoleolivetto/Downloads/instances/test.stu', 'r') as file:
    for line in file:
        student, exam = line.strip().split()
        if student in student_enrollments:
            student_enrollments[student].append(exam)
        else:
            student_enrollments[student] = [exam]

# Create a dictionary to store the number of common students for each pair of exams
common_students_dict = {}

# Iterate through the students and their enrollments
for student, exams in student_enrollments.items():
    # Generate pairs of exams for each student
    for exam1 in exams:
        for exam2 in exams:
            if exam1 != exam2:
                # Create a tuple representing the pair of exams 
                exam_pair = tuple(sorted([exam1, exam2]))
                # Initialize or update the count of common students for this pair of exams
                common_students_dict[exam_pair] = common_students_dict.get(exam_pair, 0) + 1

# Read instances from exm file --> create two lists for ExamID and the number of students enrolled
ExamID, Enrolled_stud = [], []
with open('/Users/nicoleolivetto/Downloads/instances/test.exm') as exm:
    for line in exm:
        c = (line.rstrip().split())
        if len(c) != 0:  # Skip the last empty line
            ExamID.append(c[0])
            Enrolled_stud.append(c[1])

# Parameters
E = ExamID
T = list(range(1, n_timeslots + 1))  # Include the last time slot
S = list(dict.fromkeys(StuID))  # Remove duplicates from StuID to get the IDs of the students

# Create the Gurobi environment and set parameters
env = gp.Env('/Users/nicoleolivetto/Desktop')
env.start()
env.setParam("Threads", 6)
env.setParam("Presolve", 1)
env.setParam("MIPGap", 1e-4)
env.setParam('Method', 0)
env.setParam("TimeLimit", 1000)
env.setParam("PreSparsify", 1)



# Create a Gurobi model
model = gp.Model("ETP", env=env)

# Define decision variables
x = {}
for e in E:
    for t in T:
        x[e, t] = model.addVar(vtype=GRB.BINARY, name=f"x_{e}_{t}")

# Add binary variables to represent conflicts between exams
conflict = {}
for e1 in E:
    for e2 in E:
        if e1 != e2 and (e1, e2) in common_students_dict:
            for t in T:
                conflict[e1, e2, t] = model.addVar(vtype=GRB.BINARY, name=f"conflict_{e1}_{e2}_{t}")
                             
# Add constraints to enforce that conflicting exams cannot be scheduled in the same time slot
for e1 in E:
    for e2 in E:
        if e1 != e2 and (e1, e2) in common_students_dict:
            for t in T:
                model.addConstr(x[e1, t] + x[e2, t] <= 1, name=f"conflict_{e1}_{e2}_{t}")
                

# Each exam is scheduled exactly once
for e in E:
    model.addConstr(gp.quicksum(x[e, t] for t in T) == 1, name=f"exam_scheduling_{e}")

# Define the objective function to minimize total penalty
total_penalty = gp.quicksum(2**(5 - abs(t1 - t2)) * common_students_dict[(e1, e2)] * conflict[e1, e2, t1] 
                            for e1 in E for e2 in E if e1 != e2 and (e1, e2) in common_students_dict 
                            for t1 in T for t2 in T if t1 != t2)

model.setObjective(total_penalty, GRB.MINIMIZE)

# Solve the model
model.optimize()

# Calculate the density
total_exam_pairs = len(E) * (len(E) - 1) / 2  # n choose 2
conflicting_exam_pairs = len(common_students_dict)
density = conflicting_exam_pairs / total_exam_pairs

# Print the density
print(f"Density: {density}")

#we've already defined total penalty before but now we're calculating it only for conflicting exams scheduled up to a distance of 5 time-slots)
penalty_tot = 0

# Check and print the results
if model.status == GRB.OPTIMAL or model.status == gp.GRB.SUBOPTIMAL:
    print("Feasible solution found.")
    # Initialize a dictionary to store the exam schedule
    exam_schedule = {e: None for e in E}

    # Extract and print the exam schedule
    for e in E:
        for t in T:
            if x[e, t].x > 0.5:
                exam_schedule[e] = t
                print(f"Exam {e} is scheduled at time slot {t}")

    # Iterate through the scheduled exams to calculate penalty
    for e1 in E:
        for e2 in E:
            if e1 != e2 and (e1, e2) in common_students_dict:
                # Get the time slots for the scheduled exams
                t1 = exam_schedule[e1]
                t2 = exam_schedule[e2]

                # Calculate the time difference
                time_diff = abs(t1 - t2)

                # Check if the time difference is within the acceptable range
                if time_diff >= 1 and time_diff <= 5:
                    # Calculate the penalty using the formula
                    penalty = 2**(5 - time_diff) * (common_students_dict[(e1, e2)] / len(S))
                    penalty_tot += penalty

    # Divide the total penalty by 2 to avoid double-counting (since we iterate over all pairs)
    penalty_tot /= 2

    # Print the total penalty
    print(f"Total Penalty: {penalty_tot}")

else:
    print("No optimal solution found.")


exams_at_timeslot = {t: [] for t in T}
for e in E:
    for t in T:
        if x[e, t].x > 0.5:
            exams_at_timeslot[t].append(e)


for t, exams in exams_at_timeslot.items():
    print(f"Exams scheduled at time slot {t}: {exams}")
    

instance_number = file_slo.name.split('/')[-1].split('.')[0][-2:]


with open(f"/Users/nicoleolivetto/Desktop/Base_model_sol/test.sol", "w") as file:
    file.write(f"Total Penalty: {penalty_tot}\n")
    file.write("Exam Schedule:\n")
    for e in E:
        for t in T:
            if x[e, t].x > 0.5:
                file.write(f"Exam {e} is scheduled at time slot {t}\n")




6
Set parameter Username
Set parameter LogFile to value "/Users/nicoleolivetto/Desktop"
Academic license - for non-commercial use only - expires 2024-03-17
Set parameter Threads to value 6
Set parameter Presolve to value 1
Set parameter Method to value 0
Set parameter TimeLimit to value 1000
Set parameter PreSparsify to value 1
Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (mac64[x86])

CPU model: Intel(R) Core(TM) i7-9750H CPU @ 2.60GHz
Thread count: 6 physical cores, 12 logical processors, using up to 6 threads

Optimize a model with 22 rows, 42 columns and 60 nonzeros
Model fingerprint: 0x2b97990d
Variable types: 0 continuous, 42 integer (42 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+02, 3e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 0.0000000

Explored 0 nodes (0 simplex iterations) in 0.00 seconds (0.00 work units)
Thread count was 1 (of 12 available processors)

Soluti

In [2]:
#advanced 1/2 Equity measure btb exams

# Read instances from slo file --> length of exam period
with open('/Users/nicoleolivetto/Downloads/instances/test.slo') as file_slo:
    n_timeslots = int(file_slo.readline())  # number of time-slots
    print(n_timeslots)

# Read instances from stu file --> create two lists for studentID and examIDs they are enrolled in
StuID, Ex_enr_ID = [], []
with open('/Users/nicoleolivetto/Downloads/instances/test.stu') as stu:
    for line in stu:
        b = line.rstrip().split()
        StuID.append(b[0])
        Ex_enr_ID.append(b[1])

# Define a dictionary to store the enrollments of each student
student_enrollments = {}

# Read the file and populate the student_enrollments dictionary
with open('/Users/nicoleolivetto/Downloads/instances/test.stu', 'r') as file:
    for line in file:
        student, exam = line.strip().split()
        if student in student_enrollments:
            student_enrollments[student].append(exam)
        else:
            student_enrollments[student] = [exam]

# Create a dictionary to store the number of common students for each pair of exams
common_students_dict = {}

# Iterate through the students and their enrollments
for student, exams in student_enrollments.items():
    # Generate pairs of exams for each student
    for exam1 in exams:
        for exam2 in exams:
            if exam1 != exam2:
                # Create a tuple representing the pair of exams and sort it to ensure consistency
                exam_pair = tuple(sorted([exam1, exam2]))
                # Initialize or update the count of common students for this pair of exams
                common_students_dict[exam_pair] = common_students_dict.get(exam_pair, 0) + 1

# Read instances from exm file --> create two lists for ExamID and the number of students enrolled
ExamID, Enrolled_stud = [], []
with open('/Users/nicoleolivetto/Downloads/instances/test.exm') as exm:
    for line in exm:
        c = line.rstrip().split()
        if len(c) != 0:  
            ExamID.append(c[0])
            Enrolled_stud.append(c[1])

# Parameters
E = ExamID
T = list(range(1, n_timeslots + 1))  # Include the last time slot
S = list(dict.fromkeys(StuID))  # Remove duplicates from StuID to get the IDs of the students

# Create a Gurobi model
model = gp.Model("ETP")

# Define decision variables
x = {}
for e in E:
    for t in T:
        x[e, t] = model.addVar(vtype=GRB.BINARY, name=f"x_{e}_{t}")

# Each exam is scheduled exactly once
for e in E:
    model.addConstr(gp.quicksum(x[e, t] for t in T) == 1, name=f"exam_scheduling_{e}")

# Add constraints to ensure that conflicting exams cannot be scheduled in the same time slot
for e1 in E:
    for e2 in E:
        if e1 != e2 and (e1, e2) in common_students_dict:
            for t in T:
                model.addConstr(x[e1, t] + x[e2, t] <= 1, name=f"conflict_{e1}_{e2}_{t}")

# Modify the objective function to minimize the number of students with back-to-back exams
back_to_back_sum = gp.quicksum(x[exam1, t] + x[exam2, t + 1] for student, exams in student_enrollments.items()
                                for exam1 in exams for exam2 in exams
                                if (exam1 != exam2) and ((exam1, exam2) in common_students_dict) and (t < n_timeslots - 1))

model.setObjective(back_to_back_sum, GRB.MINIMIZE)


# Solve the model
model.optimize()

# Check for back-to-back exams
back_to_back_count_after = 0
if model.status == GRB.OPTIMAL or model.status == GRB.SUBOPTIMAL:
    for student, exams in student_enrollments.items():
        for t in range(1, n_timeslots):
            for i in range(len(exams) - 1):
                exam1 = exams[i]
                exam2 = exams[i + 1]
                if (exam1, exam2) in common_students_dict:
                    if x[exam1, t].x > 0.5 and x[exam2, t + 1].x > 0.5:
                        back_to_back_count_after += 1
    print(f"Number of students with back-to-back exams after optimization: {back_to_back_count_after}")
else:
    print("No optimal solution found.")


# Print the exam schedule after minimizing students with back-to-back exams
print("Exam schedule after minimizing students with back-to-back exams:")
for e in E:
    for t in T:
        if x[e, t].x > 0.5:
            print(f"Exam {e} is scheduled at time slot {t}")

# Calculate and print the penalty
total_penalty = 0
for e1 in E:
    for e2 in E:
        if e1 != e2 and (e1, e2) in common_students_dict:
            t1 = None
            t2 = None
            for t in T:
                if x[e1, t].x > 0.5:
                    t1 = t
                if x[e2, t].x > 0.5:
                    t2 = t
            if t1 is not None and t2 is not None:
                time_diff = abs(t1 - t2)
                if time_diff >= 1 and time_diff <= 5:
                    penalty = 2**(5 - time_diff) * (common_students_dict[(e1, e2)] / len(S))
                    total_penalty += penalty
# Divide the total penalty by 2 to avoid double-counting 
total_penalty /= 2

print(f"Total Penalty: {total_penalty}")

instance_number = file_slo.name.split('/')[-1].split('.')[0][-2:]


file_path = f"/Users/nicoleolivetto/Desktop/btb_sol/btb{instance_number}.sol"

# Save the results to a .sol file
with open(file_path, "w") as file:
    
    file.write(f"Number of students with back-to-back exams after optimization: {back_to_back_count_after}\n")
    file.write(f"Total Penalty: {total_penalty}\n")
    file.write("Exam Schedule:\n")
    for e in E:
        for t in T:
            if x[e, t].x > 0.5:
                file.write(f"Exam {e} is scheduled at time slot {t}\n")

print(f"Results saved to {file_path}")



6
Set parameter Username
Academic license - for non-commercial use only - expires 2024-03-17
Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (mac64[x86])

CPU model: Intel(R) Core(TM) i7-9750H CPU @ 2.60GHz
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 22 rows, 24 columns and 60 nonzeros
Model fingerprint: 0x72a26936
Variable types: 0 continuous, 24 integer (24 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [0e+00, 0e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 0.0000000

Explored 0 nodes (0 simplex iterations) in 0.00 seconds (0.00 work units)
Thread count was 1 (of 12 available processors)

Solution count 1: 0 

Optimal solution found (tolerance 1.00e-04)
Best objective 0.000000000000e+00, best bound 0.000000000000e+00, gap 0.0000%
Number of students with back-to-back exams after optimization: 0
Exam schedule after minimizing st

In [ ]:
# Advanced model 2/2 - additional restrictions 4/4

# Read instances from slo file --> length of exam period
with open('/Users/nicoleolivetto/Downloads/instances/instance01.slo') as file_slo:
    n_timeslots = int(file_slo.readline())  # number of time-slots

# Read instances from stu file --> create two lists for studentID and examIDs they are enrolled in
StuID, Ex_enr_ID = [], []
with open('/Users/nicoleolivetto/Downloads/instances/instance01.stu') as stu:
    for line in stu:
        b = (line.rstrip().split())
        StuID.append(b[0])
        Ex_enr_ID.append(b[1])

# Define a dictionary to store the enrollments of each student
student_enrollments = {}

# Read the file and populate the student_enrollments dictionary
with open('/Users/nicoleolivetto/Downloads/instances/instance01.stu', 'r') as file:
    for line in file:
        student, exam = line.strip().split()
        if student in student_enrollments:
            student_enrollments[student].append(exam)
        else:
            student_enrollments[student] = [exam]

# Create a dictionary to store the number of common students for each pair of exams
common_students_dict = {}

# Iterate through the students and their enrollments
for student, exams in student_enrollments.items():
    # Generate pairs of exams for each student
    for exam1 in exams:
        for exam2 in exams:
            if exam1 != exam2:
                # Create a tuple representing the pair of exams and sort it to ensure consistency
                exam_pair = tuple(sorted([exam1, exam2]))
                # Initialize or update the count of common students for this pair of exams
                common_students_dict[exam_pair] = common_students_dict.get(exam_pair, 0) + 1

# Read instances from exm file --> create two lists for ExamID and the number of students enrolled
ExamID, Enrolled_stud = [], []
with open('/Users/nicoleolivetto/Downloads/instances/instance01.exm') as exm:
    for line in exm:
        c = (line.rstrip().split())
        if len(c) != 0:  
            ExamID.append(c[0])
            Enrolled_stud.append(c[1])

# Parameters
E = ExamID
T = list(range(1, n_timeslots + 1))  # Include the last time slot
S = list(dict.fromkeys(StuID))  # Remove duplicates from StuID to get the IDs of the students

# Create the Gurobi environment and set parameters
env = gp.Env('/Users/nicoleolivetto/Desktop')
env.start()
env.setParam("Threads", 6)
env.setParam("Presolve", 1)
env.setParam("MIPGap", 1e-4)
env.setParam('Method', 0)
env.setParam("TimeLimit", 1000)
env.setParam("PreSparsify", 1)

# Create a Gurobi model
model = gp.Model("ETP", env=env)

# Define decision variables
x = {}
for e in E:
    for t in T:
        x[e, t] = model.addVar(vtype=GRB.BINARY, name=f"x_{e}_{t}")

# Add binary variables to represent conflicts between exams
conflict = {}
for e1 in E:
    for e2 in E:
        if e1 != e2 and (e1, e2) in common_students_dict:
            for t in T:
                conflict[e1, e2, t] = model.addVar(vtype=GRB.BINARY, name=f"conflict_{e1}_{e2}_{t}")

# Add constraints to enforce that conflicting exams cannot be scheduled in the same time slot
for e1 in E:
    for e2 in E:
        if e1 != e2 and (e1, e2) in common_students_dict:
            for t in T:
                model.addConstr(x[e1, t] + x[e2, t] <= 1, name=f"conflict_{e1}_{e2}_{t}")

# Add constraints to prevent conflicting exams in the next 3 time slots if two consecutive time slots contain conflicts
for t in range(1, n_timeslots - 2): 
    for e1 in E:
        for e2 in E:
            if e1 != e2 and (e1, e2) in common_students_dict:
                if conflict[e1, e2, t] + conflict[e1, e2, t + 1] == 2:
                    # If conflicts occur in consecutive time slots, prevent conflicts in the next 3 time slots
                    for t_next in range(t + 2, t + 5):
                        for e1_next in E:
                            for e2_next in E:
                                if e1_next != e2_next and (e1_next, e2_next) in common_students_dict:
                                    model.addConstr(conflict[e1_next, e2_next, t_next] == 0)


# Add constraints to ensure that at most 3 consecutive time slots have conflicting exams
for t in range (n_timeslots - 2): 
    for e1 in E:
        for e2 in E:
            if e1 != e2 and (e1, e2) in common_students_dict:
                model.addConstr(gp.quicksum(conflict[e1, e2, t_prime] for t_prime in range(t, t + 3)) <= 3,
                                name=f"max_3_consecutive_conflicts_{e1}_{e2}_{t}")


# Each exam is scheduled exactly once
for e in E:
    model.addConstr(gp.quicksum(x[e, t] for t in T) == 1, name=f"exam_scheduling_{e}")
    
####
# Create a dictionary to store the count of conflicting pairs for each time slot
#conflicting_pairs_count = {t: 0 for t in T}

# Iterate through conflicting pairs and update the count for each time slot
#for e1 in E:
    #for e2 in E:
        #if e1 != e2 and (e1, e2) in common_students_dict:
            #for t in T:
                #model.addConstr(x[e1, t] + x[e2, t] <= 1 + conflicting_pairs_count[t], 
                                #name=f"conflict_{e1}_{e2}_{t}")
                # Update the count if a conflicting pair is scheduled in this time slot
                #conflicting_pairs_count[t] += 1

# Add constraints to ensure that at most 3 conflicting pairs can be scheduled in the same time slot
#for t in T:
    #model.addConstr(conflicting_pairs_count[t] <= 3, name=f"max_conflicting_pairs_{t}")

###

# Define the objective function to minimize total penalty
total_penalty = gp.quicksum(2**(5 - abs(t1 - t2)) * common_students_dict[(e1, e2)] * conflict[e1, e2, t1] 
                            for e1 in E for e2 in E if e1 != e2 and (e1, e2) in common_students_dict 
                            for t1 in T for t2 in T if t1 != t2)

# Bonus profit for no conflicting exams in 6 consecutive time slots (i used 50 but it would probably make more sense to use something like penalty/2)
bonus_profit = gp.quicksum(50 * gp.quicksum(1 - gp.quicksum(conflict[e1, e2, t] for t in T[t_index:t_index+6])
                                             for t_index in range(len(T) - 5)) 
                            for e1 in E for e2 in E if e1 != e2 and (e1, e2) in common_students_dict)

model.setObjective(total_penalty, GRB.MINIMIZE)

# Solve the model
model.optimize()

# Calculate the density
total_exam_pairs = len(E) * (len(E) - 1) / 2  
conflicting_exam_pairs = len(common_students_dict)
density = conflicting_exam_pairs / total_exam_pairs

# Print the density
print(f"Density: {density}")


penalty_tot = 0

# Check and print the results
if model.status == GRB.OPTIMAL or model.status == gp.GRB.SUBOPTIMAL:
    print("Feasible solution found.")
    # Initialize a dictionary to store the exam schedule
    exam_schedule = {e: None for e in E}

    # Extract and print the exam schedule
    for e in E:
        for t in T:
            if x[e, t].x > 0.5:
                exam_schedule[e] = t
                print(f"Exam {e} is scheduled at time slot {t}")

    # Iterate through the scheduled exams to calculate penalty
    for e1 in E:
        for e2 in E:
            if e1 != e2 and (e1, e2) in common_students_dict:
                # Get the time slots for the scheduled exams
                t1 = exam_schedule[e1]
                t2 = exam_schedule[e2]

                # Calculate the time difference
                time_diff = abs(t1 - t2)

                # Check if the time difference is within the acceptable range
                if time_diff >= 1 and time_diff <= 5:
                    # Calculate the penalty using the formula
                    penalty = 2**(5 - time_diff) * (common_students_dict[(e1, e2)] / len(S))
                    penalty_tot += penalty

    # Divide the total penalty by 2 to avoid double-counting 
    penalty_tot /= 2

    # Print the total penalty
    print(f"Total Penalty: {penalty_tot}")

else:
    print("No optimal solution found.")


exams_at_timeslot = {t: [] for t in T}
for e in E:
    for t in T:
        if x[e, t].x > 0.5:
            exams_at_timeslot[t].append(e)

# Print the exams scheduled at each time slot
for t, exams in exams_at_timeslot.items():
    print(f"Exams scheduled at time slot {t}: {exams}")
